## Cell 1: Setup and Installations

In [1]:
# Install necessary libraries
!pip install torch torchvision pandas scikit-learn datasets huggingface_hub torchmetrics tqdm -q

# --- 1. Imports and Setup ---
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from datasets import load_dataset
from tqdm.notebook import tqdm
from torchvision import transforms, models
import matplotlib.pyplot as plt

# Import the correct metrics for BINARY classification
from torchmetrics.classification import BinaryAccuracy, BinaryPrecision, BinaryRecall, BinaryF1Score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 18.1 MB/s eta 0:00:00


## Cell 2: Load and Explore the CIFAKE Dataset

In [7]:
import kagglehub
from datasets import load_dataset
import os

# --- 1. Download the dataset using KaggleHub ---
# This downloads the files and returns the local path where they are stored.
print("Downloading dataset from Kaggle...")
dataset_path = kagglehub.dataset_download("birdy654/cifake-real-and-ai-generated-synthetic-images")

print(f"Dataset downloaded to: {dataset_path}")

# --- 2. Load the data from the local folder ---
# We use the 'imagefolder' builder to load from a directory structure.
# We point it to the 'train' and 'test' sub-folders inside the downloaded path.
train_dir = os.path.join(dataset_path, 'train')
test_dir = os.path.join(dataset_path, 'test')

print(f"\nLoading images from local directories...")
dataset = load_dataset(
    'imagefolder',
    data_files={
        'train': f"{train_dir}/**", # The '/**' pattern finds all images in subfolders
        'test': f"{test_dir}/**"
    }
)

# --- 3. Verify the loaded dataset ---
print("\nDataset loaded successfully!")
print(dataset)

# You can now proceed with all your other cells for preprocessing, training, etc.
# Let's check the first training example to confirm it's correct.
print("\nExample from the training set:")
print(dataset['train'][0])

Using Colab cache for faster access to the 'cifake-real-and-ai-generated-synthetic-images' dataset.
Dataset downloaded to: /kaggle/input/cifake-real-and-ai-generated-synthetic-images

Loading images from local directories...


Resolving data files:   0%|          | 0/100000 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20000 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]


Dataset loaded successfully!
DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 20000
    })
})

Example from the training set:
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=32x32 at 0x7B0BE69A0E60>, 'label': 0}


## Cell 3: Preprocessing and DataLoaders

In [8]:
# --- 3.1 Define Image Transformations ---
def get_transforms(image_size=32): # CIFAR-10 images are 32x32
    """Returns a dictionary of data transforms."""
    imagenet_mean = [0.485, 0.456, 0.406]
    imagenet_std = [0.229, 0.224, 0.225]

    train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(image_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
    ])

    val_transforms = transforms.Compose([
        transforms.Resize(image_size), # Resize to 32
        transforms.CenterCrop(image_size), # Crop to 32
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
    ])
    return {'train': train_transforms, 'val': val_transforms}

# --- 3.2 Create DataLoaders ---
image_transforms = get_transforms()

def apply_train_transforms(batch):
    batch['pixel_values'] = [image_transforms['train'](img.convert("RGB")) for img in batch['image']]
    return batch

def apply_val_transforms(batch):
    batch['pixel_values'] = [image_transforms['val'](img.convert("RGB")) for img in batch['image']]
    return batch

dataset['train'].set_transform(apply_train_transforms)
dataset['test'].set_transform(apply_val_transforms)

def collate_fn(examples):
    pixel_values = torch.stack([example['pixel_values'] for example in examples])
    # For binary classification with BCEWithLogitsLoss, labels must be float and unsqueezed
    labels = torch.tensor([example['label'] for example in examples], dtype=torch.float32).unsqueeze(1)
    return {'pixel_values': pixel_values, 'labels': labels}

batch_size = 128
train_loader = DataLoader(dataset['train'], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(dataset['test'], batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

print(f"Data preprocessing complete.")
print(f"Created Train DataLoader with {len(train_loader)} batches of size {batch_size}")
print(f"Created Test DataLoader with {len(test_loader)} batches of size {batch_size}")

Data preprocessing complete.
Created Train DataLoader with 782 batches of size 128
Created Test DataLoader with 157 batches of size 128


## Cell 4: Model Architecture

In [9]:
# --- 4. Define the Neural Network for Image Data ---
def build_model(pretrained=True):
    """Builds a ResNet50 model for transfer learning."""
    weights = models.ResNet50_Weights.DEFAULT if pretrained else None
    model = models.resnet50(weights=weights)

    # Freeze pre-trained layers
    for param in model.parameters():
        param.requires_grad = False

    num_ftrs = model.fc.in_features

    # Replace the final layer for BINARY classification (output is 1)
    model.fc = nn.Linear(num_ftrs, 1)

    return model

# --- Instantiate the model ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = build_model().to(device)

print("Model architecture defined for binary classification:")
print("\nFinal Classifier Layer:")
print(model.fc)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 163MB/s]


Model architecture defined for binary classification:

Final Classifier Layer:
Linear(in_features=2048, out_features=1, bias=True)


## Cell 5: Training and Validation Engine

In [10]:
# --- 5. Setup for Training ---

# Use BCEWithLogitsLoss for binary classification
criterion = nn.BCEWithLogitsLoss()
# Adam optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Use Binary metrics
accuracy = BinaryAccuracy().to(device)
precision_metric = BinaryPrecision().to(device)
recall = BinaryRecall().to(device)
f1_score = BinaryF1Score().to(device)

def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    progress_bar = tqdm(loader, desc="Training")

    for batch in progress_bar:
        inputs = batch['pixel_values'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        progress_bar.set_postfix(loss=f"{loss.item():.4f}")

    return running_loss / len(loader)

def validate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    accuracy.reset()
    precision_metric.reset()
    recall.reset()
    f1_score.reset()

    with torch.no_grad():
        for batch in tqdm(loader, desc="Validating"):
            inputs = batch['pixel_values'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            # For binary metrics, apply sigmoid to the model's raw output (logits)
            preds = torch.sigmoid(outputs)
            accuracy.update(preds, labels.int())
            precision_metric.update(preds, labels.int())
            recall.update(preds, labels.int())
            f1_score.update(preds, labels.int())

    avg_loss = running_loss / len(loader)
    acc = accuracy.compute()
    prec = precision_metric.compute()
    rec = recall.compute()
    f1 = f1_score.compute()

    return avg_loss, acc, prec, rec, f1

## Cell 6: Training Execution

In [ ]:
# --- 6. Main Training Loop ---
num_epochs = 5
best_val_f1 = 0.0

for epoch in range(num_epochs):
    print(f"\n--- Epoch {epoch+1}/{num_epochs} ---")

    train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
    print(f"Epoch {epoch+1} Training Loss: {train_loss:.4f}")

    val_loss, val_acc, val_prec, val_rec, val_f1 = validate(model, test_loader, criterion)
    print(f"Epoch {epoch+1} Validation Loss: {val_loss:.4f}")
    print(f"Validation -> Accuracy: {val_acc:.4f}, Precision: {val_prec:.4f}, Recall: {val_rec:.4f}, F1-Score: {val_f1:.4f}")

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), "best_cifake_model.pth")
        print(f"New best model saved with F1-Score: {best_val_f1:.4f}")

print("\n--- Training Finished ---")


--- Epoch 1/5 ---


Training:   0%|          | 0/782 [00:00<?, ?it/s]